In [4]:
import sqlite3
import pandas as pd
from typing import Union, List, Dict, Any
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

class SQLAgent:
    """
    Role: Generate SQL queries from natural language questions using LLM
    """
    def __init__(self, model_name: str = "huyhoangt2201/llama-3.2-1b-chat-sql3-merged"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.context = """
        You are an SQL query assistant. Based on the table information below, generate an SQL query to retrieve the relevant information for the user. If the user's question is unrelated to the table, respond naturally in user's language.

        The jidouka table contains the following columns:
        id: Row identifier (int)
        tên_cải_tiến: Name of the improvement (str) 
        loại_hình_công_việc: Type of work that the improvement is intended to enhance (str)
        công_cụ: Tool used to achieve the improvement (str)
        mô_tả: Detailed description of the improvement (str)
        sản_phẩm: Output product of the improvement (str)
        tác_giả: Contributor or creator of the improvement (str)
        bộ_phận: Department of the author (str)
        số_giờ: Number of hours saved (int)
        số_công_việc_áp_dụng: Number of tasks supported (int)
        thời_điểm_ra_mắt: Launch date of the tool (str)
        thông_tin_thêm: Link to additional documentation (str)

        Return only the SQL query without any additional text. If the question is not related to the database, return "NOT_SQL_QUERY: " followed by your response.
        """
        
    def generate_query(self, question: str) -> str:
        """Generate SQL query from natural language question"""
        messages = [
            {'role': 'system', 'content': self.context},
            {'role': 'user', 'content': question}
        ]
        
        # Prepare tokenizer
        eot = "<|eot_id|>"
        eot_id = self.tokenizer.convert_tokens_to_ids(eot)
        self.tokenizer.pad_token = eot
        self.tokenizer.pad_token_id = eot_id
        
        # Generate response
        tokenized_chat = self.tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
        
        inputs = self.tokenizer(
            tokenized_chat, 
            return_tensors='pt', 
            padding=True, 
            truncation=True
        )
        
        outputs = self.model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            pad_token_id=self.tokenizer.eos_token_id,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True
        )
        
        response = self.tokenizer.decode(outputs[0])
        response = response.split('<|start_header_id|>assistant<|end_header_id|>')[1].strip()[:-10]
        
        return response.strip()

class ExecuteQueryAgent:
    """
    Role: Execute SQL queries and handle responses
    """
    def __init__(self, db_path: str):
        self.db_path = db_path
        
    def is_valid_sql_query(self, query: str) -> bool:
        """Check if the string is a valid SQL query"""
        if query.startswith("NOT_SQL_QUERY:"):
            return False
            
        # Basic SQL validation
        sql_keywords = ["SELECT", "FROM", "WHERE", "GROUP BY", "ORDER BY", "HAVING", "JOIN"]
        query_upper = query.upper()
        return any(keyword in query_upper for keyword in sql_keywords)
    
    def execute_query(self, query: str) -> Union[List[Dict[str, Any]], str]:
        """Execute SQL query and return results"""
        if not self.is_valid_sql_query(query):
            if query.startswith("NOT_SQL_QUERY:"):
                return query[13:].strip()  # Return the natural language response
            return []
            
        try:
            with sqlite3.connect(self.db_path) as conn:
                df = pd.read_sql_query(query, conn)
                return df.to_dict('records')
        except Exception as e:
            print(f"Error executing query: {e}")
            return []

class LLMAgent:
    """
    Role: Generate natural language responses from query results
    """
    def __init__(self, model_name: str = "huyhoangt2201/llama-3.2-1b-chat-sql3-merged"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        
    def generate_response(self, question: str, data_points: Union[List[Dict[str, Any]], str]) -> str:
        """Generate natural language response based on query results"""
        
        # If data_points is already a string (natural language response)
        if isinstance(data_points, str):
            return data_points
            
        # Handle empty results
        if not data_points:
            return "Tôi không tìm thấy dữ liệu phù hợp với câu hỏi của bạn. Vui lòng thử lại với câu hỏi khác."
        
        system_prompt = """You are an assistant who answers questions based on given data points.
        Requirements:
        - Answer in the same language as the user's question
        - Be concise but informative
        - If the data points are empty, answer based on general knowledge
        - Format numbers and dates appropriately
        """
        
        user_prompt = f"Question: {question}\nData points: {str(data_points)}"
        
        messages = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt}
        ]
        
        # Setup tokenizer
        eot = "<|eot_id|>"
        eot_id = self.tokenizer.convert_tokens_to_ids(eot)
        self.tokenizer.pad_token = eot
        self.tokenizer.pad_token_id = eot_id
        
        # Generate response
        tokenized_chat = self.tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
        
        inputs = self.tokenizer(
            tokenized_chat, 
            return_tensors='pt', 
            padding=True, 
            truncation=True
        )
        
        outputs = self.model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            pad_token_id=self.tokenizer.eos_token_id,
            max_new_tokens=256,
            temperature=0.7
        )
        
        response = self.tokenizer.decode(outputs[0])
        response = response.split('<|start_header_id|>assistant<|end_header_id|>')[1].strip()[:-10]
        
        return response.strip()

class MultiAgentModel:
    """
    Main class that coordinates all agents
    """
    def __init__(self, db_path: str, model_name: str = "huyhoangt2201/llama-3.2-1b-chat-sql3-merged"):
        self.sql_agent = SQLAgent(model_name)
        self.execute_agent = ExecuteQueryAgent(db_path)
        self.llm_agent = LLMAgent('phamhai/Llama-3.2-1B-Instruct-Frog')
        
    def process_question(self, question: str) -> str:
        """
        Process user question through the entire pipeline
        """
        # Step 1: Generate SQL query
        sql_query = self.sql_agent.generate_query(question)
        print(f"Generated SQL query: {sql_query}")  # For debugging
        
        # Step 2: Execute query and get results
        query_results = self.execute_agent.execute_query(sql_query)
        print(f"Query results: {query_results}")  # For debugging
        
        # Step 3: Generate natural language response
        final_response = self.llm_agent.generate_response(question, query_results)
        
        return final_response

# Example usage
if __name__ == "__main__":
    # Initialize the model
    db_path = "/kaggle/input/jidouka-database/db2.db"
    chatbot = MultiAgentModel(db_path)
    
    # Example questions
    questions = [
        "Có bao nhiêu cải tiến được thực hiện bởi Trần Thị Bình?",
        "Cho tôi biết những cải tiến nào tiết kiệm được nhiều giờ nhất?",
        "Chào bạn"
    ]
    
    # Process each question
    for question in questions:
        print(f"\nQuestion: {question}")
        response = chatbot.process_question(question)
        print(f"Response: {response}")


Question: Có bao nhiêu cải tiến được thực hiện bởi Trần Thị Bình?
Generated SQL query: SELECT COUNT(*) FROM jidouka WHERE tác_giả LIKE LOWER('%Trần Thị Bình%');
Query results: [{'COUNT(*)': 0}]
Response: Dựa vào dữ liệu được cung cấp, không có thông tin về các cải tiến mà Trần Thị Bình đã thực hiện.

Question: Cho tôi biết những cải tiến nào tiết kiệm được nhiều giờ nhất?
Generated SQL query: SELECT tên_cải_tiến, số_giờ FROM jidouka GROUP BY số_giờ ORDER BY số_giờ DESC LIMIT 1;
Query results: []
Response: Tôi không tìm thấy dữ liệu phù hợp với câu hỏi của bạn. Vui lòng thử lại với câu hỏi khác.

Question: Chào bạn
Generated SQL query: Chào bạn! Tôi không biết về các cải tiến nào có sản phẩm đầu ra là video. Bạn có thể tìm các cải tiến này trên YouTube hoặc các trang web tài liệu công cụ như GitHub hoặc StackOverflow?
Query results: []
Response: Tôi không tìm thấy dữ liệu phù hợp với câu hỏi của bạn. Vui lòng thử lại với câu hỏi khác.
